In [1]:
from yolov3 import make_new_seq_yolov3_model
from utils.utils import WeightReader
from utils.utils import preprocess_input
from utils.utils import decode_netout
from utils.utils import correct_yolo_boxes
from utils.utils import do_nms
from utils.bbox import draw_boxes
from utils.utils import get_yolo_boxes
from keras.models import Sequential
from keras.layers import  Dense

import copy
import time
import keras.backend as K
import numpy as np
import cv2
import os

Using TensorFlow backend.


In [2]:
net_h, net_w = 416, 416
obj_thresh, nms_thresh = 0.5, 0.45
anchors = [10, 13, 16, 30, 33, 23, 30, 61, 62, 45, 59, 119, 116, 90, 156, 198, 373, 326]
labels = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", \
          "boat", "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", \
          "bird", "cat", "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", \
          "backpack", "umbrella", "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", \
          "sports ball", "kite", "baseball bat", "baseball glove", "skateboard", "surfboard", \
          "tennis racket", "bottle", "wine glass", "cup", "fork", "knife", "spoon", "bowl", "banana", \
          "apple", "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza", "donut", "cake", \
          "chair", "sofa", "pottedplant", "bed", "diningtable", "toilet", "tvmonitor", "laptop", "mouse", \
          "remote", "keyboard", "cell phone", "microwave", "oven", "toaster", "sink", "refrigerator", \
          "book", "clock", "vase", "scissors", "teddy bear", "hair drier", "toothbrush"]

imgs_folder_path = 'Z:/dataset/KNU-Campus Dataset/images/20180312_171706/'
img_name = '20180312_171706_'
infer_model,seq_infer_model = make_new_seq_yolov3_model(input_shape=(416,416,3))

if not (os.path.exists('Weights/yolov3.h5')):
    weight_reader = WeightReader('yolov3.weights')
    weight_reader.load_weights(infer_model)
    infer_model.save_weights('Weights/yolov3.h5')
    seq_infer_model.load_weights('Weights/yolov3.h5', by_name=True)
else :
    infer_model.load_weights('Weights/yolov3.h5',by_name=True)
    seq_infer_model.load_weights('Weights/yolov3.h5', by_name=True)
    
output_layer_weights = [infer_model.get_layer(name='conv_81').get_weights()[0],
                        infer_model.get_layer(name='conv_93').get_weights()[0],
                        infer_model.get_layer(name='conv_105').get_weights()[0]]

In [24]:
def get_grad(yolo_num,row,col,out_idx):
    prev_layer_shape = [(13,13,1024),(26,26,512),(52,52,256)]
    result=np.zeros(shape=prev_layer_shape[yolo_num])
    result[row,col,:] = output_layer_weights[yolo_num][0,0,:,out_idx]
    return result

In [32]:
start_time = time.time()

for i in range(19):
    a=get_grad(yolo_num = 2, row = i, col = i, out_idx = i)
    a+=get_grad(yolo_num = 2, row = i, col = i, out_idx =i+1)
print(time.time()-start_time)

0.0852351188659668


In [34]:
prev_feature=[]
for img_num in range(900):
#     print(img_num)
#     if(img_num >= 2):
#         img_num=2
        
    img_num2str=str(img_num)
    while(len(img_num2str)<4):
        img_num2str='0'+img_num2str

    img = cv2.imread(imgs_folder_path+img_name+img_num2str+'.jpg')
    image_h, image_w, _ = img.shape
    
    process_image = preprocess_input(img, net_h, net_w)
    
    if img_num == 0:
        yolos = infer_model.predict(process_image)
        prev_feature = copy.deepcopy(yolos[:3])
        yolos = yolos[:3]
    else:
        start_time = time.time()
        yolos = infer_model.predict(process_image)
        update_layer = yolos[3:]
        
        l_r = 1
        gradients=[None]*3
        count=[0]*3
        print(len(rows))
        for i in range(len(rows)):
            row = rows[i]
            col = cols[i]
            b = bs[i]
            class_num = class_nums[i]
            yolo_num = yolo_nums[i]
            if (count[yolo_num]==0):
                gradients[yolo_num] = get_grad(yolo_num = yolo_num, row = row, col = col, out_idx = 85*b +4) 
                gradients[yolo_num] += get_grad(yolo_num = yolo_num, row = row, col = col, out_idx = 85*b +5+ class_num)
            else:
                gradients[yolo_num] += get_grad(yolo_num = yolo_num, row = row, col = col, out_idx = 85*b +4)
                gradients[yolo_num] += get_grad(yolo_num = yolo_num, row = row, col = col, out_idx = 85*b +5+ class_num)
                
            count[yolo_num] += 1
            
        for i in range(3):
            if (count[i]!=0):
                gradients[i] /= float(count[i])
                update_layer[i] = update_layer[i] + l_r * gradients[i]
        
        yolos = seq_infer_model.predict([update_layer[0],update_layer[1],update_layer[2],prev_feature[0],prev_feature[1],prev_feature[2]])
        prev_feature = copy.deepcopy(yolos[:3])
        yolos = yolos[:3]
        print(time.time()-start_time)
        
        
    
    #yolos = infer_model.predict(process_image)
    boxes = []

    for i in range(len(yolos)):
        # decode the output of the network
        yolo_anchors = anchors[(2 - i) * 6:(3 - i) * 6]
        boxes += decode_netout(yolos[i][0], yolo_anchors, obj_thresh, net_h, net_w, i)
    #print(boxes)
    # correct the sizes of the bounding boxes
    correct_yolo_boxes(boxes, image_h, image_w, net_h, net_w)

    # suppress non-maximal boxes
    do_nms(boxes, nms_thresh)

    # draw bounding boxes on the image using labels
    rows, cols, bs, class_nums, yolo_nums = draw_boxes(img, boxes, labels, obj_thresh, debug = False)
    cv2.imwrite('outputs/new_seq_attend/'+img_name+img_num2str+'_detected.jpg',cv2.resize(img,(1280,720)))
    cv2.imshow('video with bboxes', cv2.resize(img,(1280,720)))
    cv2.waitKey(6)
#     print()

19
0.18950510025024414
19
0.1894993782043457
21
0.1874995231628418
19
0.17747116088867188
21
0.18448805809020996
20
0.18449115753173828
19
0.17947816848754883


KeyboardInterrupt: 